In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from os.path import join
from scipy.stats import pearsonr

import warnings
warnings.filterwarnings(action='ignore')

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

directory = r'C:\Users\vince\Downloads\ABM-Research'
cambodia_file = r'Game Data\joinedDataCambodia.csv'
vietnam_file = r'Game Data\joinedDataVietnam.csv'
merged_file = r'Game Data\mergedData.csv'

clusters_folder="Clusters"

cambodia_csv = pd.read_csv(os.path.join(directory, cambodia_file), engine="python")
vietnam_csv = pd.read_csv(os.path.join(directory, vietnam_file), engine='python')
merged_csv = pd.read_csv(os.path.join(directory, merged_file), engine='python')

In [3]:
# 4 clusters based on cell type
# # labels_full=labels
labels_full_vietnam=np.array(pd.read_csv(join(directory, clusters_folder, "labels_4_cell_type_vietnam.csv")))
labels_full_vietnam=np.array([arr[0] for arr in labels_full_vietnam])
centers_vietnam=np.array(pd.read_csv(join(directory, clusters_folder, "centers_4_cell_type_vietnam.csv")))

# labels_2_vietnam=np.array(pd.read_csv(join(directory, clusters_folder, "labels_2_cell_type_vietnam.csv")))
# labels_2_vietnam=np.array([arr[0] for arr in labels_2_vietnam])
# centers_2_vietnam=np.array(pd.read_csv(join (directory, clusters_folder, "centers_2_cell_type_vietnam.csv")))

labels_full_cambodia=np.array(pd.read_csv(join(directory, clusters_folder, "labels_4_cell_type_cambodia.csv")))
labels_full_cambodia=np.array([arr[0] for arr in labels_full_cambodia])
centers_cambodia=np.array(pd.read_csv(join (directory, clusters_folder, "centers_4_cell_type_cambodia.csv")))

# labels_2_cambodia=np.array(pd.read_csv(join(directory, clusters_folder, "labels_2_cell_type_cambodia.csv")))
# labels_2_cambodia=np.array([arr[0] for arr in labels_2_cambodia])
# centers_2_cambodia=np.array(pd.read_csv(join (directory, clusters_folder, "centers_2_cell_type_cambodia.csv")))

labels_full_merged=np.array(pd.read_csv(join(directory, clusters_folder, "labels_4_cell_type_merged.csv")))
labels_full_merged=np.array([arr[0] for arr in labels_full_merged])
centers_merged=np.array(pd.read_csv(join (directory, clusters_folder, "centers_4_cell_type_merged.csv")))

labels_2_merged=np.array(pd.read_csv(join(directory, clusters_folder, "labels_2_cell_type_merged.csv")))
labels_2_merged=np.array([arr[0] for arr in labels_2_merged])
centers_2_merged=np.array(pd.read_csv(join (directory, clusters_folder, "centers_2_cell_type_merged.csv")))

hs_labels={"Vietnam": [3], "Cambodia": [1], "Merged": [1]}
nch_labels={"Vietnam":  [1, 2], "Cambodia": [2, 3], "Merged": [2, 3]}
base_labels={"Vietnam":  [0], "Cambodia": [0], "Merged": [0]}
strategy_labels={"Vietnam":  ['Neutral', 'NCH (Moderate)', 'NCH (High)', 'Heavy Spray'], 
                 "Cambodia": ['Neutral', 'Heavy Spray', 'NCH (Moderate)', 'NCH (High)'],
                 "Merged": ['Neutral', 'Heavy Spray', 'NCH (High)', 'NCH (Moderate)']}

def set_country(curr_str):
    global curr_csv
    global hs_label
    hs_label=hs_labels[curr_str]
    global nch_label
    nch_label=nch_labels[curr_str]
    global base_label
    base_label=base_labels[curr_str]
    global strategy_label
    strategy_label=strategy_labels[curr_str]
    global labels_full
    global centers
    # for 2 clusters
    global labels_2, centers_2, nch_2, hs_2

    nch_2=0
    hs_2=1

    if curr_str == "Merged":
        labels_full= labels_full_merged
        centers = centers_merged
        curr_csv = merged_csv
        labels_2=labels_2_merged
        centers_2=centers_2_merged
        nch_2=0
        hs_2=1

    elif curr_str == "Vietnam":
        labels_full= labels_full_vietnam
        centers = centers_vietnam
        curr_csv = vietnam_csv
        labels_2=labels_2_merged[:4480*4]
        centers_2=centers_2_merged[:4480*4]
        # labels_2=labels_2_vietnam
        # centers_2=centers_2_vietnam
        # nch_2=1
        # hs_2=0
    elif curr_str == "Cambodia":
        labels_full= labels_full_cambodia
        centers = centers_cambodia
        curr_csv = cambodia_csv
        labels_2=labels_2_merged[4480*4:]
        centers_2=centers_2_merged[4480*4:]
        # labels_2=labels_2_cambodia
        # centers_2=centers_2_cambodia
        # nch_2=1
        # hs_2=0

def calculate_score_csv(row_num, curr_csv):
    plot=np.array(curr_csv.iloc[row_num, 40:76]).reshape(6,6)
    cell_yield=[5, 0, 6, 10]
    # print(plot)
    scores=np.zeros((6, 6))
    if curr_csv['numNCH'][row_num] == 0:
        subsidy = 0
    else:
        subsidy = curr_csv['sumSubsidies'][row_num]/curr_csv['numNCH'][row_num]

    cell_yield[1]=subsidy

    for row in range(6):
        for col in range(6):
            if plot[row, col]==1:
                for vert in range(max(0, row-2), min(6, row+3)):
                    # print(row, max(0, row-2), min(6, row+3))
                    for horiz in range(max(0, col-2), min(6, col+3)):
                        if plot[vert, horiz]==1:
                            continue
                        scores[vert, horiz]+=2

    for row in range(6):
        for col in range(6):
            if plot[row, col] == 3:
                for vert in range(max(0, row-1), min(6, row+2)):
                    # print(row, max(0, row-2), min(6, row+3))
                    for horiz in range(max(0, col-1), min(6, col+2)):
                        scores[vert, horiz]=0

    for row in range(6):
        for col in range(6):
            scores[row, col] += cell_yield[plot[row, col]]
    
    # print(scores)

    for row in range(6):
        for col in range(6):
            if scores[row, col]>15:
                scores[row, col]=15

    full_yield=[0]*4
    for i in range(4):
        plot_temp= np.rot90(scores, i+1)
        corner=plot_temp[:3, :3]
        full_yield[i]=int(np.sum(corner))
    return full_yield

def calculate_score(board, subsidy):
    plot=np.array(board).reshape(6,6)
    cell_yield=[5, 0, 6, 10]
    # print(plot)
    scores=np.zeros((6, 6))

    cell_yield[1]=subsidy

    for row in range(6):
        for col in range(6):
            if plot[row, col]==1:
                for vert in range(max(0, row-2), min(6, row+3)):
                    # print(row, max(0, row-2), min(6, row+3))
                    for horiz in range(max(0, col-2), min(6, col+3)):
                        if plot[vert, horiz]==1:
                            continue
                        scores[vert, horiz]+=2

    for row in range(6):
        for col in range(6):
            if plot[row, col] == 3:
                for vert in range(max(0, row-1), min(6, row+2)):
                    # print(row, max(0, row-2), min(6, row+3))
                    for horiz in range(max(0, col-1), min(6, col+2)):
                        scores[vert, horiz]=0

    for row in range(6):
        for col in range(6):
            scores[row, col] += cell_yield[plot[row, col]]
    
    # print(scores)

    for row in range(6):
        for col in range(6):
            if scores[row, col]>15:
                scores[row, col]=15

    full_yield=[0]*4
    for i in range(4):
        plot_temp= np.rot90(scores, i+1)
        corner=plot_temp[:3, :3]
        full_yield[i]=int(np.sum(corner))
    return full_yield


def calculate_net_yield(board):
    # plot=np.array(curr_csv.iloc[row_num, 40:76]).reshape(6,6)
    plot = np.array(board).reshape(6, 6)
    cell_yield=[5, 0, 6, 10]
    # print(plot)
    scores=np.zeros((6, 6))

    for row in range(6):
        for col in range(6):
            if plot[row, col]==1:
                for vert in range(max(0, row-2), min(6, row+3)):
                    # print(row, max(0, row-2), min(6, row+3))
                    for horiz in range(max(0, col-2), min(6, col+3)):
                        if plot[vert, horiz]==1:
                            continue
                        scores[vert, horiz]+=2

    for row in range(6):
        for col in range(6):
            if plot[row, col] == 3:
                for vert in range(max(0, row-1), min(6, row+2)):
                    for horiz in range(max(0, col-1), min(6, col+2)):
                        scores[vert, horiz]=0

    for row in range(6):
        for col in range(6):
            scores[row, col] += cell_yield[plot[row, col]]
    
    # print(scores)

    for row in range(6):
        for col in range(6):
            if scores[row, col]>15:
                scores[row, col]=15

    full_yield=[0]*4
    for i in range(4):
        plot_temp= np.rot90(scores, i+1)
        corner=plot_temp[:3, :3]
        full_yield[i]=int(np.sum(corner))
    return full_yield


def calculate_nch(board):
    # plot = np.array(curr_csv.iloc[row_num, 40:76]).reshape(6, 6)
    plot = np.array(board).reshape(6, 6)
    # cell_yield = [5, 0, 6, 10]
    nch_nums = [0, 0, 0]
    # print(plot)
    scores = np.zeros((6, 6))

    for row in range(6):
        for col in range(6):
            if plot[row, col] == 1:
                for vert in range(max(0, row-2), min(6, row+3)):
                    # print(row, max(0, row-2), min(6, row+3))
                    for horiz in range(max(0, col-2), min(6, col+3)):
                        if plot[vert, horiz] == 1 or plot[vert, horiz] == 3:
                            continue
                        scores[vert, horiz] += 2

    for row in range(6):
        for col in range(6):
            if plot[row, col] == 3:
                for vert in range(max(0, row-1), min(6, row+2)):
                    for horiz in range(max(0, col-1), min(6, col+2)):
                        nch_nums[0]+=scores[vert, horiz]
                        scores[vert, horiz] = 0

    for row in range(6):
        for col in range(6):
            nch_nums[1] += scores[row, col]
            if plot[row, col] == 1:
                nch_nums[2]+=1

    #[cancelled, bonus, num cells]
    return nch_nums
    
cells_map=np.zeros((6, 6)).tolist()
for i in range(6):
    for j in range(6):
        if i<3 and j<3:
            cells_map[i][j]=1
        elif i>=3 and j<3:
            cells_map[i][j]=4
        elif i<3 and j>=3:
            cells_map[i][j]=2
        else:
            cells_map[i][j]=3

In [10]:
set_country("Vietnam")
# ["Score", "Strategy Switches", "HS to NCH", "Farmers in NCH", "Number of NCH Cells", "Number of HS Cells"]
size=6
data=[[0]*11 for _ in range(size)]
totals=[[0]*11 for _ in range(size)]

for i in range(len(curr_csv)):
    if curr_csv["Game"][i]=='GP':
        continue
    curr_plot=np.array(curr_csv.iloc[i, 40:76]).reshape(6,6)
    subsidy=curr_csv["Subsidy"][i]

    # score
    round_score=calculate_score_csv(i, curr_csv)
    data[0][subsidy]+=sum(round_score)
    totals[0][subsidy]+=1

    if curr_csv["Round"][i]!=1:
        #strategy switches
        totals[1][subsidy]+=1
        totals[2][subsidy]+=1
        for j in range(4):
            data[1][subsidy]+=labels_2[i*4+j]!=labels_2[(i-1)*4+j]
        #strategy switches hs to nch
            data[2][subsidy]+=(labels_2[(i-1)*4+j]==hs_2 and labels_2[i*4+j]==nch_2) 

    # number of farmers in NCH
    totals[3][subsidy]+=1
    for j in range(4):
        data[3][subsidy]+=labels_2[i*4+j]==nch_2

# num nch and hs cells
    totals[4][subsidy]+=1
    data[4][subsidy]+=curr_csv["numNCH"][i]
    
    totals[5][subsidy]+=1
    data[5][subsidy]+=curr_csv["numHS"][i]

for i in range(size):
    for j in range(11):
        data[i][j]/=totals[i][j]
data=np.array(data)

max_vals=[460, 4, 4, 4, 36, 36]
# data
for i in range(size):
    data[i]=data[i]/max_vals[i]

pd.DataFrame([max_vals]).to_csv(os.path.join(directory, "calibration_max_vals.csv"), index=False, header=False)

In [11]:
headers=["Score", "Strategy Switches", "HS to NCH", "Farmers in NCH", "Number of NCH Cells", "Number of HS Cells"]
df=pd.DataFrame()
for i in range(size):
    temp=[]
    for j in range(11):
        temp.append(data[i][j])
    df[headers[i]]=temp
df.to_csv(os.path.join(directory, "calibration_data_vietnam.csv"), index=False, header=False)

In [45]:
set_country("Cambodia")
size=6
data=[[0]*11 for _ in range(size)]
totals=[[0]*11 for _ in range(size)]

for i in range(len(curr_csv)):
    if curr_csv["Game"][i]=='GP':
        continue
    curr_plot=np.array(curr_csv.iloc[i, 40:76]).reshape(6,6)
    subsidy=curr_csv["Subsidy"][i]

    # score
    round_score=calculate_score_csv(i, curr_csv)
    data[0][subsidy]+=sum(round_score)
    totals[0][subsidy]+=1

    if curr_csv["Round"][i]!=1:
        #strategy switches
        totals[1][subsidy]+=1
        totals[2][subsidy]+=1
        for j in range(4):
            data[1][subsidy]+=labels_2[i*4+j]!=labels_2[(i-1)*4+j]
        #strategy switches hs to nch
            data[2][subsidy]+=(labels_2[(i-1)*4+j]==hs_2 and labels_2[i*4+j]==nch_2) 

    # number of farmers in NCH
    totals[3][subsidy]+=1
    for j in range(4):
        data[3][subsidy]+=labels_2[i*4+j]==nch_2

# num nch and hs cells
    totals[4][subsidy]+=1
    data[4][subsidy]+=curr_csv["numNCH"][i]
    
    totals[5][subsidy]+=1
    data[5][subsidy]+=curr_csv["numHS"][i]

for i in range(size):
    for j in range(11):
        data[i][j]/=totals[i][j]
data=np.array(data)

max_vals=[460, 4, 4, 4, 36, 36]
# data
for i in range(size):
    data[i]=data[i]/max_vals[i]

pd.DataFrame([max_vals]).to_csv(os.path.join(directory, "calibration_max_vals.csv"), index=False, header=False)

In [46]:
headers=["Score", "Strategy Switches", "HS to NCH", "Farmers in NCH", "Number of NCH Cells", "Number of HS Cells"]
# indices=["Scale Factor", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
# df=pd.DataFrame(columns=headers)
df=pd.DataFrame()
for i in range(size):
    temp=[]
    # temp=[max_vals[i]]
    for j in range(11):
        temp.append(data[i][j])
    df[headers[i]]=temp

df.to_csv(os.path.join(directory, "calibration_data_cambodia.csv"), index=False, header=False)